In [1]:
import pandas as pd
from utils.busca_ceps import cep_to_coords


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_seq_items', None)

# md ENEM

In [2]:
df_enem_raw = pd.read_csv("dados/inputs/RESULTADOS_2024.csv", sep=";", encoding="latin1")

df_enem_raw.info()  # 4.3M linhas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4332944 entries, 0 to 4332943
Data columns (total 42 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   NU_SEQUENCIAL           int64  
 1   NU_ANO                  int64  
 2   CO_ESCOLA               float64
 3   CO_MUNICIPIO_ESC        float64
 4   NO_MUNICIPIO_ESC        object 
 5   CO_UF_ESC               float64
 6   SG_UF_ESC               object 
 7   TP_DEPENDENCIA_ADM_ESC  float64
 8   TP_LOCALIZACAO_ESC      float64
 9   TP_SIT_FUNC_ESC         float64
 10  CO_MUNICIPIO_PROVA      int64  
 11  NO_MUNICIPIO_PROVA      object 
 12  CO_UF_PROVA             int64  
 13  SG_UF_PROVA             object 
 14  TP_PRESENCA_CN          int64  
 15  TP_PRESENCA_CH          int64  
 16  TP_PRESENCA_LC          int64  
 17  TP_PRESENCA_MT          int64  
 18  CO_PROVA_CN             float64
 19  CO_PROVA_CH             float64
 20  CO_PROVA_LC             float64
 21  CO_PROVA_MT             float64

### Filtrando linhas e colunas

In [3]:
# Remover linhas que nao tenham codigo escola
df = df_enem_raw.dropna(subset="CO_ESCOLA")  # 1.5M linhas

# Remove quem faltou qualquer dia
cols = ["TP_PRESENCA_CN", "TP_PRESENCA_CH", "TP_PRESENCA_LC", "TP_PRESENCA_MT"]
df = df[df[cols].eq(1).all(axis=1)]  # 1.19M linhas

# Remove escolas com menos de 5 provas
df = df.groupby('CO_ESCOLA').filter(lambda g: len(g) >= 5)  # 1.18M linhas

df = df[["CO_ESCOLA", "NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT", "NU_NOTA_REDACAO"]]

df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1186962 entries, 0 to 4332941
Data columns (total 6 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   CO_ESCOLA        1186962 non-null  float64
 1   NU_NOTA_CN       1186962 non-null  float64
 2   NU_NOTA_CH       1186962 non-null  float64
 3   NU_NOTA_LC       1186962 non-null  float64
 4   NU_NOTA_MT       1186962 non-null  float64
 5   NU_NOTA_REDACAO  1186962 non-null  float64
dtypes: float64(6)
memory usage: 63.4 MB


### Agrupando alunos de mesma escola

In [4]:
df_escolas = df.groupby("CO_ESCOLA", as_index=False).agg("mean")  # 26_302 escolas

df_escolas["nota_objetiva"] = df_escolas[["NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT"]].mean(axis=1)

df_escolas["nota_enem"] = df_escolas["nota_objetiva"] + df_escolas["NU_NOTA_REDACAO"]

df_escolas["cod_escola"] = df_escolas["CO_ESCOLA"].astype(int)  # estava com .0 no final

df_escolas.shape

(26302, 9)

In [5]:
df_enem = df_escolas[["cod_escola", "nota_enem"]]

df_enem.head()

,cod_escola,nota_enem
0,11000058,1381.738462
1,11000198,1240.234043
2,11000252,1289.052941
3,11000260,1280.931061
4,11000317,922.709375


### Salvando

In [6]:

df_enem.to_csv("dados/temporarios/01-escolas_enem.csv", index=False)

# Dados Consultores

In [7]:
df_consultores_raw = pd.read_excel("dados/inputs/local_consultores.xlsx",
                       usecols=["Consultor", "CEP"])

df_consultores_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Consultor  27 non-null     object
 1   CEP        24 non-null     object
dtypes: object(2)
memory usage: 564.0+ bytes


In [8]:
df = df_consultores_raw.dropna().copy()

df["CEP"] = df["CEP"].str.replace("-", "")


In [9]:
df_consultor_coords = await cep_to_coords(df, "CEP")


Temos 24 CEPs diferentes a consultar
Nao foi necessario baixar novas coods


In [10]:
print(df_consultor_coords.info())
display(df_consultor_coords.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Consultor  24 non-null     object 
 1   lat        24 non-null     float64
 2   lon        24 non-null     float64
dtypes: float64(2), object(1)
memory usage: 708.0+ bytes
None


,Consultor,lat,lon
0,Arthur Lincoln,-22.96851,-43.18552
1,Ana Claudia,-23.21074,-45.85701
2,Elina Maciel,-22.87912,-47.03464
3,Galdino Espindola,-22.91447,-43.05093
4,Niedna Leite,-23.59439,-46.61307


### Salvando

In [11]:
df_consultor_coords.to_csv("dados/temporarios/01-consultor_coords.csv", index=False)
